# PlasmaPy Optical Thomson Scattering Tutorial

[thomson]: https://docs.plasmapy.org/en/latest/ad/diagnostics/thomson.html
[spectral-density]: https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.thomson.spectral_density.html#plasmapy.diagnostics.thomson.spectral_density
[froula]: https://www.sciencedirect.com/book/9780123748775/plasma-scattering-of-electromagnetic-radiation

Thomson scattering is scattering of an incident photon of of a charged particle (ion or electron) or off of waves in a plasma. The spectrum of the scattered light encodes information about the distribution function of the plasma, and is commonly used to measure the plasma density, temperature, and flow velocity.

<center><img src="images/ots_setup.png" width="700"></center>


The [thomson.spectral_density][spectral-density] function calculates the [spectral density function S(k,w)][froula], which is one of several terms that determine the scattered power spectrum for the Thomson scattering of a probe laser beam by a plasma. In particular, this function calculates $S(k,w)$ for the case of a plasma consisting of one or more ion species and electron populations under the assumption that all of the ion species and the electron fluid have Maxwellian velocity distribution functions and that the combined plasma is quasi-neutral. In this regime, the spectral density is given by the equation:

\begin{equation}
S(k,\omega) = \sum_e \frac{2\pi}{k} \bigg |1 - \frac{\chi_e}{\epsilon} \bigg |^2 f_{e0,e}\bigg ( \frac{\omega}{k} \bigg ) + \sum_i \frac{2\pi Z_i}{k} \bigg | \frac{\chi_e}{\epsilon} \bigg |^2 f_{i0, i} \bigg ( \frac{\omega}{k} \bigg )
\end{equation}

where $\chi_e$ is the electron component susceptibility of the plasma and $\epsilon = 1 + \sum_e \chi_e + \sum_i \chi_i$ is the total plasma dielectric function (with $\chi_i$ being the ion component of the susceptibility), $Z_i$ is the charge of each ion, $k$ is the scattering wavenumber, $\omega$ is the scattering frequency, and the functions $f_{e0,e}$ and $f_{i0,i}$ are the Maxwellian velocity distributions for the electrons and ion species respectively.

<center><img src="images/collective_vs_noncollective_ots.png"></center>

Thomson scattering can be either non-collective (the scattered spectrum is a linear sum of the light scattered by individual particles) or collective (the scattered spectrum is dominated by scattering off of collective plasma waves). The [thomson.spectral_density][spectral-density] function can be used in both cases. These regimes are delineated by the dimensionless constant $\alpha$:

\begin{equation}
\alpha = \frac{1}{k \lambda_{De}}
\end{equation}

where $\lambda_{De}$ is the Debye length. $\alpha > 1$ corresponds to collective scattering, while $\alpha < 1$ corresponds to non-collective scattering. Depending on which of these regimes applies, fitting the scattered spectrum can provide the electron (and sometimes ion) density and temperature. Doppler shifting of the spectrum can also provide a measurement of the drift velocity of each plasma species.

For a detailed explanation of the underlying physics (and derivations of these expressions), see ["Plasma Scattering of Electromagnetic Radiation" by Froula et al.][froula]

In [ ]:
import sys

if 'google.colab' in str(get_ipython()):
    if 'plasmapy' not in sys.modules:
        !pip install plasmapy==2023.5.1 requests==2.27.1
    
%matplotlib inline

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from plasmapy.diagnostics import thomson

[PlasmaPy documentation]:https://docs.plasmapy.org/en/latest/ad/diagnostics/thomson.html
[spectral_density]: https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.thomson.spectral_density.html#plasmapy.diagnostics.thomson.spectral_density

This notebook provides a simple example using the [spectral_density] function in PlasmaPy to model Thomson scattering. More examples and documentation can be found in the [PlasmaPy documentation]


The first step is to define the scattering geometry, the wavelength of the probe laser, and the wavelength range of the detector

In [ ]:
# The probe wavelength can in theory be anything, but in practice integer frequency multiples of the Nd:YAG wavelength
# 1064 nm are used (532 corresponds to a frequency-doubled probe beam from such a laser).
probe_wavelength = 532 * u.nm

# Array of wavelengths over which to calculate the spectral distribution

wavelengths = (
    np.linspace(probe_wavelength.value - 30, probe_wavelength.value + 30, num=5000)
    * u.nm
)


# The scattering geometry is defined by unit vectors for the orientation of the probe laser beam (probe_n) and
# the path from the scattering volume (where the measurement is made) to the detector (scatter_n).
# These can be setup for any experimental geometry - this geometry is typical on the OMEGA laser facility.
probe_vec = np.array([1, 0, 0])
scattering_angle = np.deg2rad(63)
scatter_vec = np.array([np.cos(scattering_angle), np.sin(scattering_angle), 0])

[spectral_density]: https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.thomson.spectral_density.html#plasmapy.diagnostics.thomson.spectral_density
Now we will specify a set of plasma parameters, call the [spectral_density] function, and plot the results

In [ ]:
n = 2e17 * u.cm ** -3
T_e = 12 * u.eV
T_i = 10 * u.eV
ions = 'H+'

alpha, Skw = thomson.spectral_density(
    wavelengths,
    probe_wavelength,
    n,
    T_e=T_e,
    T_i=T_i,
    ions = ions,
    probe_vec=probe_vec,
    scatter_vec=scatter_vec,
)

For convenience later, let's define a function that will make a nice plot of the spectral density function

In [ ]:
# This function makes a plot of the Thomson scattering spectrum
def plot_result(wavelengths, alpha, Skw, iaw_range=np.array([528,536])*u.nm):
    # Estimate the max of the EPW signal for plotting
    a = np.argmin(np.abs(iaw_range[0]-wavelengths))
    b = np.argmin(np.abs(iaw_range[1]-wavelengths))
    iaw_max = np.max(Skw.value)
    epw_max = max([np.max(Skw[0:a].value), np.max(Skw[b:].value)])
    
    fig, ax = plt.subplots(ncols=2, figsize=(12, 6))
    fig.subplots_adjust(wspace=0.2)
    
    if alpha > 1:
        fig.suptitle(f"alpha = {alpha:.1f} (collective scattering)")
    else:
        fig.suptitle(f"alpha = {alpha:.1f} (noncollective scattering)")

    for x in ax:
        x.set_xlabel("Wavelength (nm)")
        x.set_ylabel("Skw")
        x.axvline(x=probe_wavelength.value, color="red", label="Probe wavelength")
    ax[0].tick_params(axis='both', which='major', labelsize=12)
    ax[0].set_xlim(520, 545)
    ax[0].set_ylim(0, 1.2*epw_max)
    ax[0].set_title("Electron Plasma Wave", fontsize=14)
    ax[0].plot(wavelengths.value, Skw)
    ax[0].legend(loc='upper left', fontsize=12)

    ax[1].tick_params(axis='both', which='major', labelsize=12)
    ax[1].set_xlim(530, 534)
    ax[1].set_ylim(0, 1.2*iaw_max)
    ax[1].set_title("Ion Acoustic Wave", fontsize=14)
    ax[1].plot(wavelengths.value, Skw)
    ax[1].legend(loc='upper left', fontsize=12);



Now we can use this function to plot the results

In [ ]:
plot_result(wavelengths, alpha, Skw)

The spectral density function can also include multiple populations of ions and electrons, each of which can have a separate drift velocity in three dimensions. Here's a more complex example.

In [ ]:
n = 2e17 * u.cm ** -3

efract = np.array([0.5, 0.5])
T_e = np.array([20, 6]) * u.eV
electron_vel = np.array([[0,0,0], [1000, 0, 0]])*u.km/u.s


ifract = np.array([0.2, 0.8])
ions = ["He-4 1+", "C-12 5+"]
T_i = np.array([10, 15]) * u.eV
ion_vel = np.array([[300,0,0], [-900, 0, 0]])*u.km/u.s

alpha, Skw = thomson.spectral_density(
    wavelengths,
    probe_wavelength,
    n,
    T_e=T_e,
    T_i=T_i,
    efract = efract,
    ifract = ifract, 
    ions = ions,
    electron_vel=electron_vel,
    ion_vel=ion_vel,
    probe_vec=probe_vec,
    scatter_vec=scatter_vec,
)

plot_result(wavelengths, alpha, Skw)

[spectral density function example notebook]: https://docs.plasmapy.org/en/latest/notebooks/diagnostics/thomson.html
For more examples, see the [spectral density function example notebook]. 

# PlasmaPy can also fit Thomson scattering spectra using the `lmfit` module! 

[spectral_density_model()]: https://docs.plasmapy.org/en/latest/api/plasmapy.diagnostics.thomson.spectral_density_model.html#plasmapy.diagnostics.thomson.spectral_density_model
[Thomson scattering fitting example notebook]: https://docs.plasmapy.org/en/latest/notebooks/diagnostics/thomson_fitting.html

See the [spectral_density_model()] and the associated [Thomson scattering fitting example notebook]. 